# Human Presence Detection Training using Tensorflow


#### Click on "Kernel -> Restart & Run All" to start
#### Click on "Kernel -> Shutdown" before closing the window

In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true;
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import yaml
import os
from collections import OrderedDict 
config_data = OrderedDict()
if 'CONFIG_FILE' not in os.environ:
    os.environ['CONFIG_FILE'] = './' #config file path
config_file = os.path.join(os.environ['CONFIG_FILE'], "config.txt")
if os.path.isfile(config_file):
    with open(config_file) as fp:
        config_data = yaml.load(fp.read(), Loader=yaml.FullLoader)
else:
    config_data["DATA_DIR"] = None
    config_data["Log_dir"] = None
    config_data["FILTER_TRAIN_KEYWORD"] = "marvin,sheila,on,off,up,down,go,stop,left,right,yes,learn,visual,follow,no,cat,dog,bird,tree,house,bed,wow,happy,zero,one,two,three,four,five,six,seven,eight,nine,forward,backward"
    config_data["TRAIN_KEYWORD"] = "seven,marvin,on,happy"

In [3]:
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog
data_dir = None

class SelectPathButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""
    def __init__(self):
        super(SelectPathButton, self).__init__(layout=widgets.Layout(width='20%', height='30px'))
        
        self.description = "Select dataset path"
        self.on_click(self.select_path)

    @staticmethod
    def select_path(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askdirectory(initialdir = "/")
        if b.files != ():
            global data_dir
            data_dir = b.files
            path.value = data_dir
            config_data["DATA_DIR"] = data_dir
            with open(config_file, 'w') as outf:
                yaml.dump(config_data, outf, default_flow_style=False)
                outf.close()
        

button = SelectPathButton()
if config_data["DATA_DIR"] is not None:
    path = widgets.Text(layout=widgets.Layout(width='30%'), value=config_data["DATA_DIR"])
    data_dir = config_data["DATA_DIR"]
else:
    path = widgets.Text(layout=widgets.Layout(width='30%'))

widgets.HBox([path, button])

### Training Configuration and Start Training
#### User can customize the training parameters by changing the following values. 
1. __train directory:__ The volume which is shared by host to docker environment should contain a train logs directory. Using the "Select train directory" button select that path from docker. The path of shared folder in docker environment will same be as that in host machine.
2. __Filter Training keywords:__ Name of all classes available in dataset direcory except background_noice.
3. __Training keyword names:__ Name of keywords you want to train the model with.

In [4]:
from ipywidgets import widgets
from IPython.display import display
from tkinter import Tk, filedialog
train_dir = None
class SelectPathButton(widgets.Button):
    """A file widget that leverages tkinter.filedialog."""
    def __init__(self):
        super(SelectPathButton, self).__init__(layout=widgets.Layout(width='20%', height='30px'))
        # Create the button.
        self.description = "Select train directory"
        self.on_click(self.select_path)

    @staticmethod
    def select_path(b):
        """Generate instance of tkinter.filedialog.

        Parameters
        ----------
        b : obj:
            An instance of ipywidgets.widgets.Button 
        """
        # Create Tk root
        root = Tk()
        # Hide the main window
        root.withdraw()
        # Raise the root to the top of all windows.
        root.call('wm', 'attributes', '.', '-topmost', True)
        # List of selected fileswill be set to b.value
        b.files = filedialog.askdirectory(initialdir = "/")
        if b.files != ():
            global train_dir
            train_dir = b.files
            train_path.value = train_dir
            config_data["Log_dir"] = train_dir
            with open(config_file, 'w') as outf:
                yaml.dump(config_data, outf, default_flow_style=False)
                outf.close()

train_button = SelectPathButton()
if config_data["Log_dir"] is not None:
    train_path = widgets.Text(layout=widgets.Layout(width='30%'), value=config_data["Log_dir"])
    train_dir = config_data["Log_dir"]
else:
    train_path = widgets.Text(layout=widgets.Layout(width='30%'))

widgets.HBox([train_path, train_button])

In [5]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import subprocess 

style = {'description_width': '100px'}
layout = {'width': '800px'}

print('\033[1m' + "Hit ENTER on textbox after entering the keyword name !")
print('\033[0m')
filter_keywords = widgets.Text(description = 'Filter keywords',value =config_data["FILTER_TRAIN_KEYWORD"], disabled=False, style=style, layout=layout)
training_keywords = widgets.Text(description = 'Training keywords',value =config_data["TRAIN_KEYWORD"], disabled=False, style=style, layout=layout)
run_filter_button = widgets.Button(description="Run Filter Training", layout=widgets.Layout(width='25%', height='25px'))
run_training_button = widgets.Button(description="Run Keyword Training", layout=widgets.Layout(width='25%', height='25px'))
training_out = widgets.Output()

global f_keywords, t_keywords, owd
owd = os.getcwd()
os.environ['owd'] = os.getcwd()
f_keywords = filter_keywords.value
t_keywords = training_keywords.value
    
def callback_filter_key(sender):
    global f_keywords
    f_keywords = filter_keywords.value

def callback_training_key(sender):
    global t_keywords
    t_keywords = training_keywords.value

def on_filter_button_clicked(_):
    with training_out:
        global f_keywords, t_keywords
        clear_output()
        os.chdir(os.environ['owd'])
        if train_dir is not None and data_dir is not None:
            run_filter_button.disabled=True
            run_training_button.disabled=True
            print('\033[1m' + "To stop the running training, click Kernel->Interrupt option from menubar")
            print('\033[0m')
            os.environ['FILTER_TRAIN_KEYWORD'] = f_keywords
            os.environ['TRAIN_KEYWORD'] = t_keywords
            os.environ['DATA_DIR'] = data_dir
            os.environ['LOG_DIR'] = train_dir
            os.environ['FILTER_TRAIN_DIR'] = os.environ['LOG_DIR']+'/set8_seven.filter'
            os.environ['TRAIN_DIR'] = os.environ['LOG_DIR']+'/set_prefilter'
            os.environ['COMMON_OPT'] = '--model_architecture=tinyvgg_conv -sample_rate=8000 --downsample=1 --no_prefilter_bias --clip_duration_ms=1040 --time_shift_ms=140.0 --window_size_ms=32.0 --window_stride_ms=16.0 --dct_coefficient_count=64 --background_volume=0.5'
            !python train.py $COMMON_OPT --wanted_words=$FILTER_TRAIN_KEYWORD --silence_percentage=5 --unknown_percentage=5 --how_many_training_steps=20000,20000,10000 --learning_rate=0.01,0.001,0.0001 --batch_size=100 --train_dir=$FILTER_TRAIN_DIR --data_dir=$DATA_DIR --summaries_dir=$LOG_DIR/cmd_seven.filter --data_url=  --optimizer=Adam
            run_training_button.disabled=False
            run_filter_button.disabled=False
            with open(config_file, 'w') as outf:
                config_data["FILTER_TRAIN_KEYWORD"] = f_keywords
                config_data["TRAIN_KEYWORD"] = t_keywords
                yaml.dump(config_data, outf, default_flow_style=False)
                outf.close()
        else:
            print("Please select dataset and training directory")

def on_training_button_clicked(_):
    with training_out:
        global f_keywords, t_keywords
        clear_output()
        os.chdir(os.environ['owd'])
        if train_dir is not None and data_dir is not None:
            run_training_button.disabled=True
            run_filter_button.disabled=True
            print('\033[1m' + "To stop the running training, click Kernel->Interrupt option from menubar")
            print('\033[0m')
            os.environ['FILTER_TRAIN_KEYWORD'] = f_keywords
            os.environ['TRAIN_KEYWORD'] = t_keywords
            os.environ['DATA_DIR'] = data_dir
            os.environ['LOG_DIR'] = train_dir
            os.environ['FILTER_TRAIN_DIR'] = os.environ['LOG_DIR']+'/set8_seven.filter'
            os.environ['TRAIN_DIR'] = os.environ['LOG_DIR']+'/set_prefilter'
            os.environ['NETWORK']='tinyvgg_conv'
            os.environ['TRAIN_OPT']="--optimizer=Adam --set_prefilter="+os.path.join(os.environ['FILTER_TRAIN_DIR'],"tinyvgg_conv.ckpt-50000")+" --lock_prefilter"
            os.environ['COMMON_OPT']="--model_architecture=tinyvgg_conv --sample_rate=8000 --downsample=1 --no_prefilter_bias --clip_duration_ms=1040 --time_shift_ms=140.0 --window_size_ms=32.0 --window_stride_ms=16.0 --dct_coefficient_count=64 --background_volume=0.5"
            !python train.py $COMMON_OPT --wanted_words=$TRAIN_KEYWORD --silence_percentage=100 --unknown_percentage=100 --how_many_training_steps=20000,20000,10000 --learning_rate=0.01,0.001,0.0001 --batch_size=100 --train_dir=$TRAIN_DIR --data_dir=$DATA_DIR --summaries_dir=$LOG_DIR/cmd_seven $TRAIN_OPT
            run_training_button.disabled=False
            run_filter_button.disabled=False
            with open(config_file, 'w') as outf:
                config_data["FILTER_TRAIN_KEYWORD"] = f_keywords
                config_data["TRAIN_KEYWORD"] = t_keywords
                yaml.dump(config_data, outf, default_flow_style=False)
                outf.close()
        else:
            print("Please select dataset and training directory")

run_filter_button.on_click(on_filter_button_clicked)
run_training_button.on_click(on_training_button_clicked)
filter_keywords.on_submit(callback_filter_key)
training_keywords.on_submit(callback_training_key)

widgets.VBox([widgets.HBox([widgets.VBox([filter_keywords, training_keywords,run_filter_button,run_training_button])]), training_out])

Hit ENTER on textbox after entering the keyword name !



### Training Status

#### Tensorboard
TensorBoard provides us with a suite of web applications that help us to inspect and understand the TensorFlow runs and graphs. Currently, it provides five types of visualizations: scalars, images, audio, histograms, and graphs.

In [6]:
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import subprocess, os

button_descriptions  = {False: "Stop Tensorboard", True: "Launch Tensorboard"}
p = None
out = widgets.Output()

global toggle_button , opt
opt = ['Filter training', 'Keyword training']
toggle_button = widgets.ToggleButtons(
    options=opt,
    description='',
    disabled=False,
    button_style='')

def button_action(value):
    global p, toggle_button
    os.chdir(os.environ['owd'])
    if train_dir is not None:
        if button.description == "Launch Tensorboard":
            if toggle_button.value == opt[0]:
                os.chdir(os.path.join(train_dir,'cmd_seven.filter/train'))
            elif toggle_button.value == opt[1]:
                os.chdir(os.path.join(train_dir,'cmd_seven/train'))
            toggle_button.disabled=True
            if [file for file in glob.glob("*.tfevents*")] != []:
                cmd = ["tensorboard", "--logdir=./"]
                p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
                for line in p.stderr:
                    if line.decode("utf8").find("(Press CTRL+C to quit)") > 0:
                        print(line.decode("utf8").split("(Press CTRL+C")[0])
                        state = False
                        break
                    else:
                        print(line.decode("utf8"))
                        state = True
            else:
                print("No event file found in train directory")
                state = True
        else:
            if p is not None:
                print("Tensorboard stopped")
                p.kill()
                state = True
                toggle_button.disabled=False
        
        value.owner.description = button_descriptions[state]
    else:
        print("Please select train directory")


state = True

button = widgets.ToggleButton(state, description = button_descriptions[state], layout=widgets.Layout(width='20%', height='50px'))
button.observe(button_action, "value")
display(toggle_button, button)

ToggleButtons(options=('Filter training', 'Keyword training'), value='Filter training')

ToggleButton(value=True, description='Launch Tensorboard', layout=Layout(height='50px', width='20%'))

### Model Freezing
#### Click the below button to generate the frozen pb from checkpoint and graph.pbtxt files. The pb file will easily pass through the sensAI tool. The generated pb file will be copied into the shared train directory so that it can be used in host environment.

In [7]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import tensorflow as tf
import os, glob

freeze_button = widgets.Button(description="Model Freezing", layout=widgets.Layout(width='20%', height='50px'))
freeze_out = widgets.Output()

def on_button_clicked(_):
    with freeze_out:
        clear_output()
        os.chdir(owd)
        if train_dir is not None:
            print("Model freezing with the latest checkpoint")
            !python train.py $COMMON_OPT --freeze --train_dir=$TRAIN_DIR --data_dir=$DATA_DIR --wanted_words=$TRAIN_KEYWORD
            !python genpb.py --ckpt_dir $LOG_DIR/set_prefilter
            os.chdir(os.environ['LOG_DIR']+'/set_prefilter')
            if "SAVE_FILE_PATH" in os.environ:
                !cp *.pb $SAVE_FILE_PATH
            else:
                !cp *.pb $owd
                print("Save File path is not set!, coping the fozenpb to root direcory !")
            os.chdir(owd)
            print("Model .pb generated and copied to train directory")
        else:
            print("Please select train directory")
             

# linking button and function together using a button's method
freeze_button.on_click(on_button_clicked)
# displaying button and its output together
widgets.VBox([freeze_button, freeze_out])